In [1]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

In [2]:
role = get_execution_role()
prefix = 'sagemaker-sklearn-linearlearner'
my_region = boto3.session.Session().region_name # set the region of the instance

In [5]:
my_region

'us-east-2'

In [9]:
bucket_name = 'task-bucket-2021'
s3 = boto3.client('s3')

s3.download_file(bucket_name, 'data/dataset.csv', '../data/dataset.csv')

In [19]:
data_df = pd.read_csv('../data/dataset.csv', sep=';', na_values='NA')
data_df.head()

,uuid,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,account_status,account_worst_status_0_3m,account_worst_status_12_24m,account_worst_status_3_6m,account_worst_status_6_12m,age,avg_payment_span_0_12m,avg_payment_span_0_3m,merchant_category,merchant_group,has_paid,max_paid_inv_0_12m,max_paid_inv_0_24m,name_in_email,num_active_div_by_paid_inv_0_12m,num_active_inv,num_arch_dc_0_12m,num_arch_dc_12_24m,num_arch_ok_0_12m,num_arch_ok_12_24m,num_arch_rem_0_12m,num_arch_written_off_0_12m,num_arch_written_off_12_24m,num_unpaid_bills,status_last_archived_0_24m,status_2nd_last_archived_0_24m,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours,worst_status_active_inv
0,63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0,NaN,20,12.692308,8.333333,Dietary supplements,Health & Beauty,True,31638.0,31638.0,no_match,0.153846,2,0,0,13,14,0,0.0,0.0,2,1,1,1,1,1,1,0,0,0,178839,9.653333,1.0
1,0e961183-8c15-4470-9a5e-07a1bd207661,0.0,0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,1.0,1.0,50,25.833333,25.000000,Books & Magazines,Entertainment,True,13749.0,13749.0,F+L,0.000000,0,0,0,9,19,3,0.0,0.0,0,1,1,1,1,2,2,0,0,0,49014,13.181389,NaN
2,d8edaae6-4368-44e0-941e-8328f203e64e,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,22,20.000000,18.000000,Diversified entertainment,Entertainment,True,29890.0,29890.0,L1+F,0.071429,1,0,0,11,0,3,0.0,0.0,1,1,1,1,1,2,2,0,0,0,124839,11.561944,1.0
3,0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,4.687500,4.888889,Diversified entertainment,Entertainment,True,40040.0,40040.0,F1+L,0.031250,1,0,0,31,21,0,0.0,0.0,1,1,1,1,1,1,1,0,0,0,324676,15.751111,1.0
4,c8f8b835-5647-4506-bf15-49105d8af30b,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,25,13.000000,13.000000,Electronic equipment & Related accessories,Electronics,True,7100.0,7100.0,F+L,0.000000,0,0,0,1,0,0,0.0,0.0,0,1,0,0,1,1,1,0,0,0,7100,12.698611,NaN


In [20]:
pd.set_option('display.max_columns', 100)

In [21]:
data_df.describe()

,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,account_status,account_worst_status_0_3m,account_worst_status_12_24m,account_worst_status_3_6m,account_worst_status_6_12m,age,avg_payment_span_0_12m,avg_payment_span_0_3m,max_paid_inv_0_12m,max_paid_inv_0_24m,num_active_div_by_paid_inv_0_12m,num_active_inv,num_arch_dc_0_12m,num_arch_dc_12_24m,num_arch_ok_0_12m,num_arch_ok_12_24m,num_arch_rem_0_12m,num_arch_written_off_0_12m,num_arch_written_off_12_24m,num_unpaid_bills,status_last_archived_0_24m,status_2nd_last_archived_0_24m,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours,worst_status_active_inv
count,89976.000000,9.997600e+04,88140.000000,88140.000000,88140.000000,40661.000000,45603.000000,45603.000000,33215.000000,42274.000000,39626.000000,99976.000000,76140.000000,50671.000000,99976.000000,99976.000000,77037.000000,99976.000000,99976.000000,99976.000000,99976.000000,99976.000000,99976.000000,81898.000000,81898.000000,99976.000000,99976.000000,99976.000000,99976.000000,99976.000000,99976.000000,99976.000000,99976.000000,99976.000000,99976.000000,9.997600e+04,99976.000000,30461.000000
mean,0.014315,1.225515e+04,0.223043,5.044622,0.286896,1.331292,1.042168,1.172905,1.337348,1.185291,1.253142,36.016284,17.971473,14.989786,9203.654217,11215.120819,0.114840,0.599404,0.061745,0.059364,7.275826,6.369799,0.469443,0.000122,0.000159,2.141564,0.923202,0.827519,0.744299,0.800582,1.052233,1.226164,4.035429,10816.065386,6542.895325,3.920880e+04,15.329780,1.121762
std,0.118786,3.548148e+04,5.808117,22.863971,2.929910,26.482299,0.202713,0.420142,0.575043,0.443309,0.505559,13.001306,12.751066,10.297420,13512.167229,15256.414937,0.293483,1.550026,0.374691,0.366224,16.030369,15.350244,1.360349,0.011049,0.014407,6.300978,0.569530,0.622812,0.634912,0.719946,0.786121,0.833502,163.934564,26463.972170,19041.223585,9.064929e+04,5.031360,0.343660
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000278,1.000000
25%,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,25.000000,10.800000,8.400000,2000.000000,3350.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,2.600000e+03,11.622708,1.000000
50%,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.152082,1.000000,1.000000,1.000000,1.000000,1.000000,34.000000,14.909091,13.000000,6052.000000,7580.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.599500e+04,15.792778,1.000000
75%,0.000000,4.937250e+03,0.000000,0.000000,0.000000,0.662952,1.000000,1.000000,2.000000,1.000000,1.000000,45.000000,21.000000,18.285714,11380.000000,13783.000000,0.100000,1.000000,0.000000,0.000000,7.000000,6.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,0.000000,9029.750000,85.000000,4.384425e+04,19.542014,1.000000
max,1.000000,1.128775e+06,365.000000,365.000000,97.000000,3914.000000,4.000000,4.000000,4.000000,4.000000,4.000000,100.000000,260.000000,87.000000,279000.000000,538500.000000,9.000000,47.000000,17.000000,13.000000,261.000000,313.000000,42.000000,1.000000,2.000000,182.000000,5.000000,5.000000,5.000000,3.000000,5.000000,5.000000,36479.000000,571475.000000,341859.000000,2.962870e+06,23.999722,3.000000


In [29]:
train_test_df = data_df[data_df['default'].notna()]

submit_df = data_df[data_df['default'].isna()]
submit_df = submit_df.drop(columns='default')

In [32]:
data_df.shape, train_test_df.shape, submit_df.shape

((99976, 43), (89976, 43), (10000, 42))

In [34]:
data_df.nunique()

uuid                                   99976
default                                    2
account_amount_added_12_24m            23721
account_days_in_dc_12_24m                127
account_days_in_rem_12_24m               282
account_days_in_term_12_24m               64
account_incoming_debt_vs_paid_0_24m    23674
account_status                             4
account_worst_status_0_3m                  4
account_worst_status_12_24m                4
account_worst_status_3_6m                  4
account_worst_status_6_12m                 4
age                                       79
avg_payment_span_0_12m                  7939
avg_payment_span_0_3m                   2256
merchant_category                         57
merchant_group                            12
has_paid                                   2
max_paid_inv_0_12m                     12497
max_paid_inv_0_24m                     12932
name_in_email                              8
num_active_div_by_paid_inv_0_12m         861
num_active

In [52]:
label = 'default'

variables = [x for x in data_df.columns if x != 'uuid' and x != label]

numeric_cols = [
    'account_amount_added_12_24m',
    'account_days_in_dc_12_24m',
    'account_days_in_rem_12_24m',
    'account_days_in_term_12_24m',
    'account_incoming_debt_vs_paid_0_24m',
    'age',
    'avg_payment_span_0_12m',
    'avg_payment_span_0_3m',
    'max_paid_inv_0_12m',
    'max_paid_inv_0_24m',
    'num_active_div_by_paid_inv_0_12m',
    'num_active_inv',
    'num_arch_dc_0_12m',
    'num_arch_dc_12_24m',
    'num_arch_ok_0_12m',
    'num_arch_ok_12_24m',
    'num_arch_rem_0_12m',
    'num_arch_written_off_0_12m',
    'num_arch_written_off_12_24m',
    'num_unpaid_bills',
    'recovery_debt',
    'sum_capital_paid_account_0_12m',
    'sum_capital_paid_account_12_24m',
    'sum_paid_inv_0_12m',
    'time_hours'    
]

categorical_cols = [x for x in variables if x not in numeric_cols]

In [53]:
len(variables), len(numeric_cols), len(categorical_cols)

(41, 25, 16)

In [54]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(train_test_df, test_size=0.2)

train.shape, test.shape

((71980, 43), (17996, 43))

In [55]:
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numeric_transformer = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

categorical_transformer = make_pipeline(
    SimpleImputer(strategy="constant", fill_value=-1),
    OneHotEncoder(handle_unknown="ignore"),
)

In [56]:
preprocessor = ColumnTransformer(
        transformers=[
            ("numeric", numeric_transformer, numeric_cols),
            ("categorical", categorical_transformer, categorical_cols),
        ]
    )

In [57]:
preprocessor.fit(train[variables])

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['account_amount_added_12_24m',
                                  'account_days_in_dc_12_24m',
                                  'account_days_in_rem_12_24m',
                                  'account_days_in_term_12_24m',
                                  'account_incoming_debt_vs_paid_0_24m', 'age',
                                  'avg_payment_span_0_12m',
                                  'avg_payment_sp...
                                  'account_worst_status_12_24m',
                                  'account_worst_status_3_6m',
                                  'account_worst_status_6_12m',
                       

In [58]:
preprocessor.transform(train[variables])

<71980x161 sparse matrix of type '<class 'numpy.float64'>'
	with 2951180 stored elements in Compressed Sparse Row format>

In [59]:
!pwd

/home/ec2-user/SageMaker/task/notebook
